# Task 1: Exploratory Data Analysis and Data Preprocessing

## Intelligent Complaint Analysis for Financial Services

This notebook performs comprehensive exploratory data analysis and preprocessing of the CFPB complaint dataset for the RAG-powered chatbot project.

**Objectives:**
- Load and explore the CFPB complaint dataset
- Analyze complaint distributions across financial products
- Perform text analysis on complaint narratives
- Clean and filter data for the specified products
- Prepare data for embedding and vector store creation

In [1]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
import os
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import textstat
from wordcloud import WordCloud
import re
from datetime import datetime
import zipfile
import requests
from io import BytesIO

# Configure plotting
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Set up NLTK
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
    
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

print("Libraries imported successfully!")
print(f"Current working directory: {os.getcwd()}")

ModuleNotFoundError: No module named 'plotly'

## 1. Data Loading and Initial Exploration

We'll load the CFPB complaint dataset and perform initial exploration to understand the structure and content of the data.

In [ ]:
# Load the CFPB complaint dataset
# URL for the Consumer Complaint Database
data_url = "https://files.consumerfinance.gov/ccdb/complaints.csv.zip"

# Check if data already exists
data_path = "../data/complaints.csv"
if not os.path.exists(data_path):
    print("Downloading CFPB complaint dataset...")
    
    # Download the dataset
    response = requests.get(data_url)
    if response.status_code == 200:
        # Extract the CSV from the zip file
        with zipfile.ZipFile(BytesIO(response.content)) as zip_file:
            zip_file.extractall("../data/")
        print("Dataset downloaded and extracted successfully!")
    else:
        print(f"Failed to download dataset. Status code: {response.status_code}")
else:
    print("Dataset already exists locally.")

# Load the dataset
print("Loading dataset...")
df = pd.read_csv(data_path, low_memory=False)

print(f"Dataset loaded successfully!")
print(f"Dataset shape: {df.shape}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
# Initial data exploration
print("=== DATASET OVERVIEW ===")
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print("\n=== FIRST FEW ROWS ===")
print(df.head())

print("\n=== DATA TYPES ===")
print(df.dtypes)

print("\n=== MISSING VALUES ===")
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_values,
    'Missing Percentage': missing_percentage
}).sort_values('Missing Count', ascending=False)
print(missing_df[missing_df['Missing Count'] > 0])

print("\n=== BASIC STATISTICS ===")
print(df.describe(include='all'))

## 2. Product Distribution Analysis

Let's analyze the distribution of complaints across different financial products, with focus on our target products: Credit card, Personal loan, Buy Now Pay Later (BNPL), Savings account, and Money transfers.

In [ ]:
# Define our target products
target_products = [
    'Credit card',
    'Personal loan', 
    'Buy Now, Pay Later (BNPL)',
    'Savings account',
    'Money transfers'
]

# Check unique products in the dataset
print("=== ALL PRODUCTS IN DATASET ===")
product_counts = df['Product'].value_counts()
print(product_counts)

print(f"\nTotal unique products: {len(product_counts)}")
print(f"Total complaints: {len(df)}")

# Map similar product names to our target products
product_mapping = {
    'Credit card or prepaid card': 'Credit card',
    'Payday loan, title loan, or personal loan': 'Personal loan',
    'Payday loan, title loan, personal loan, or advance loan': 'Personal loan',
    'Money transfer, virtual currency, or money service': 'Money transfers',
    'Money transfers': 'Money transfers',
    'Checking or savings account': 'Savings account',
    'Bank account or service': 'Savings account',
    'Credit card': 'Credit card',
    'Personal loan': 'Personal loan',
    'Buy Now, Pay Later (BNPL)': 'Buy Now, Pay Later (BNPL)',
    'Savings account': 'Savings account'
}

# Apply mapping and filter
df['Product_mapped'] = df['Product'].map(product_mapping)
df['Product_mapped'] = df['Product_mapped'].fillna('Other')

print("\n=== MAPPED PRODUCT DISTRIBUTION ===")
mapped_counts = df['Product_mapped'].value_counts()
print(mapped_counts)

# Filter for target products
df_filtered = df[df['Product_mapped'].isin(target_products)].copy()
print(f"\nFiltered dataset shape: {df_filtered.shape}")
print(f"Percentage of original data retained: {len(df_filtered)/len(df)*100:.2f}%")

In [ ]:
# Visualize product distribution
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('All Products (Top 10)', 'Target Products Distribution', 
                   'Target Products Over Time', 'Complaints by Company Type'),
    specs=[[{"type": "bar"}, {"type": "pie"}],
           [{"type": "scatter"}, {"type": "bar"}]]
)

# Top 10 products
top_products = product_counts.head(10)
fig.add_trace(
    go.Bar(x=top_products.index, y=top_products.values, name="All Products"),
    row=1, col=1
)

# Target products pie chart
target_counts = df_filtered['Product_mapped'].value_counts()
fig.add_trace(
    go.Pie(labels=target_counts.index, values=target_counts.values, name="Target Products"),
    row=1, col=2
)

# Time series of complaints for target products
df_filtered['Date received'] = pd.to_datetime(df_filtered['Date received'])
df_filtered['Year'] = df_filtered['Date received'].dt.year
yearly_complaints = df_filtered.groupby(['Year', 'Product_mapped']).size().reset_index(name='Count')

for product in target_products:
    product_data = yearly_complaints[yearly_complaints['Product_mapped'] == product]
    fig.add_trace(
        go.Scatter(x=product_data['Year'], y=product_data['Count'], 
                  name=product, mode='lines+markers'),
        row=2, col=1
    )

# Company distribution (top 10)
company_counts = df_filtered['Company'].value_counts().head(10)
fig.add_trace(
    go.Bar(x=company_counts.values, y=company_counts.index, 
           orientation='h', name="Top Companies"),
    row=2, col=2
)

fig.update_layout(height=800, title_text="Complaint Distribution Analysis")
fig.update_xaxes(tickangle=45, row=1, col=1)
fig.show()

print("=== TARGET PRODUCTS SUMMARY ===")
for product in target_products:
    count = target_counts.get(product, 0)
    percentage = (count / len(df_filtered)) * 100
    print(f"{product}: {count:,} complaints ({percentage:.1f}%)")

## 3. Text Analysis of Complaint Narratives

Now let's analyze the consumer complaint narratives to understand text characteristics and identify patterns.

In [ ]:
# Analyze complaint narratives
print("=== NARRATIVE ANALYSIS ===")

# Check for missing narratives
narrative_col = 'Consumer complaint narrative'
total_complaints = len(df_filtered)
complaints_with_narratives = df_filtered[narrative_col].notna().sum()
complaints_without_narratives = total_complaints - complaints_with_narratives

print(f"Total complaints: {total_complaints:,}")
print(f"Complaints with narratives: {complaints_with_narratives:,} ({complaints_with_narratives/total_complaints*100:.1f}%)")
print(f"Complaints without narratives: {complaints_without_narratives:,} ({complaints_without_narratives/total_complaints*100:.1f}%)")

# Filter out complaints without narratives
df_with_narratives = df_filtered[df_filtered[narrative_col].notna()].copy()
print(f"\nFiltered dataset with narratives: {len(df_with_narratives):,}")

# Analyze text length
df_with_narratives['narrative_length'] = df_with_narratives[narrative_col].str.len()
df_with_narratives['word_count'] = df_with_narratives[narrative_col].str.split().str.len()
df_with_narratives['sentence_count'] = df_with_narratives[narrative_col].str.split('.').str.len()

print("\n=== TEXT LENGTH STATISTICS ===")
print("Character count:")
print(df_with_narratives['narrative_length'].describe())
print("\nWord count:")
print(df_with_narratives['word_count'].describe())
print("\nSentence count:")
print(df_with_narratives['sentence_count'].describe())

# Identify very short and very long narratives
print("\n=== EXTREME CASES ===")
short_threshold = 50  # characters
long_threshold = 5000  # characters

very_short = df_with_narratives[df_with_narratives['narrative_length'] <= short_threshold]
very_long = df_with_narratives[df_with_narratives['narrative_length'] >= long_threshold]

print(f"Very short narratives (≤{short_threshold} chars): {len(very_short):,} ({len(very_short)/len(df_with_narratives)*100:.1f}%)")
print(f"Very long narratives (≥{long_threshold} chars): {len(very_long):,} ({len(very_long)/len(df_with_narratives)*100:.1f}%)")

# Show examples
print("\n=== EXAMPLE SHORT NARRATIVE ===")
if len(very_short) > 0:
    print(very_short[narrative_col].iloc[0])
    
print("\n=== EXAMPLE LONG NARRATIVE (truncated) ===")
if len(very_long) > 0:
    print(very_long[narrative_col].iloc[0][:500] + "...")

In [ ]:
# Visualize text length distributions
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Character count distribution
axes[0, 0].hist(df_with_narratives['narrative_length'], bins=50, alpha=0.7, color='skyblue')
axes[0, 0].set_title('Distribution of Character Count')
axes[0, 0].set_xlabel('Character Count')
axes[0, 0].set_ylabel('Frequency')

# Word count distribution
axes[0, 1].hist(df_with_narratives['word_count'], bins=50, alpha=0.7, color='lightgreen')
axes[0, 1].set_title('Distribution of Word Count')
axes[0, 1].set_xlabel('Word Count')
axes[0, 1].set_ylabel('Frequency')

# Box plot of word count by product
df_with_narratives.boxplot(column='word_count', by='Product_mapped', ax=axes[1, 0])
axes[1, 0].set_title('Word Count by Product')
axes[1, 0].set_xlabel('Product')
axes[1, 0].set_ylabel('Word Count')

# Scatter plot: word count vs character count
axes[1, 1].scatter(df_with_narratives['word_count'], df_with_narratives['narrative_length'], 
                  alpha=0.1, s=1)
axes[1, 1].set_title('Word Count vs Character Count')
axes[1, 1].set_xlabel('Word Count')
axes[1, 1].set_ylabel('Character Count')

plt.tight_layout()
plt.show()

# Word frequency analysis
print("=== WORD FREQUENCY ANALYSIS ===")

# Function to clean and tokenize text
def clean_and_tokenize(text):
    if pd.isna(text):
        return []
    # Convert to lowercase and remove special characters
    text = re.sub(r'[^\w\s]', '', text.lower())
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words and len(token) > 2]
    return tokens

# Sample a subset for word frequency analysis (for performance)
sample_size = min(10000, len(df_with_narratives))
sample_df = df_with_narratives.sample(n=sample_size, random_state=42)

# Tokenize all narratives
all_tokens = []
for narrative in sample_df[narrative_col]:
    tokens = clean_and_tokenize(narrative)
    all_tokens.extend(tokens)

# Get most common words
word_freq = Counter(all_tokens)
top_words = word_freq.most_common(30)

print("Top 30 most common words:")
for word, count in top_words:
    print(f"{word}: {count}")

# Create word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_freq)

plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Complaint Narratives')
plt.show()

## 4. Data Cleaning and Preprocessing

Now let's clean the text narratives to improve embedding quality and prepare the final dataset.

In [ ]:
# Text cleaning function
def clean_complaint_text(text):
    """
    Clean complaint text for better embedding quality
    """
    if pd.isna(text):
        return None
    
    # Convert to string and lowercase
    text = str(text).lower()
    
    # Remove common boilerplate text
    boilerplate_phrases = [
        "i am writing to file a complaint",
        "i would like to file a complaint",
        "this is a complaint about",
        "dear sir or madam",
        "to whom it may concern",
        "i am contacting you regarding",
        "i am writing this letter to",
        "xxxx", "xx/xx/xxxx"
    ]
    
    for phrase in boilerplate_phrases:
        text = text.replace(phrase, "")
    
    # Remove excessive whitespace and normalize
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    
    # Remove very short texts (less than 20 characters)
    if len(text) < 20:
        return None
    
    return text

# Apply cleaning
print("=== CLEANING COMPLAINT NARRATIVES ===")
print(f"Before cleaning: {len(df_with_narratives)} complaints")

df_with_narratives['cleaned_narrative'] = df_with_narratives[narrative_col].apply(clean_complaint_text)

# Remove rows with None values after cleaning
df_cleaned = df_with_narratives.dropna(subset=['cleaned_narrative']).copy()

print(f"After cleaning: {len(df_cleaned)} complaints")
print(f"Removed: {len(df_with_narratives) - len(df_cleaned)} complaints")

# Compare original vs cleaned text length
df_cleaned['original_length'] = df_cleaned[narrative_col].str.len()
df_cleaned['cleaned_length'] = df_cleaned['cleaned_narrative'].str.len()

print("\n=== TEXT LENGTH COMPARISON ===")
print("Original text length:")
print(df_cleaned['original_length'].describe())
print("\nCleaned text length:")
print(df_cleaned['cleaned_length'].describe())

# Show example of cleaning
print("\n=== CLEANING EXAMPLE ===")
idx = 0
print("Original:")
print(df_cleaned[narrative_col].iloc[idx])
print("\nCleaned:")
print(df_cleaned['cleaned_narrative'].iloc[idx])

In [ ]:
# Prepare final dataset
print("=== PREPARING FINAL DATASET ===")

# Select relevant columns for the RAG system
columns_to_keep = [
    'Complaint ID',
    'Product_mapped',
    'Issue',
    'Sub-issue',
    'Company',
    'State',
    'Date received',
    'cleaned_narrative',
    'original_length',
    'cleaned_length'
]

# Create final dataset
final_df = df_cleaned[columns_to_keep].copy()
final_df = final_df.rename(columns={
    'Product_mapped': 'Product',
    'cleaned_narrative': 'Consumer_complaint_narrative'
})

print(f"Final dataset shape: {final_df.shape}")
print(f"Final dataset columns: {final_df.columns.tolist()}")

# Final statistics by product
print("\n=== FINAL DATASET STATISTICS BY PRODUCT ===")
product_stats = final_df.groupby('Product').agg({
    'Consumer_complaint_narrative': 'count',
    'cleaned_length': ['mean', 'median', 'std']
}).round(2)

product_stats.columns = ['Count', 'Mean_Length', 'Median_Length', 'Std_Length']
print(product_stats)

# Save the cleaned dataset
output_path = "../data/filtered_complaints.csv"
final_df.to_csv(output_path, index=False)
print(f"\n✅ Cleaned dataset saved to: {output_path}")

# Summary report
print("\n" + "="*50)
print("SUMMARY REPORT")
print("="*50)
print(f"Original dataset size: {len(df):,} complaints")
print(f"After product filtering: {len(df_filtered):,} complaints ({len(df_filtered)/len(df)*100:.1f}%)")
print(f"After narrative filtering: {len(df_with_narratives):,} complaints ({len(df_with_narratives)/len(df)*100:.1f}%)")
print(f"Final cleaned dataset: {len(final_df):,} complaints ({len(final_df)/len(df)*100:.1f}%)")
print(f"\nTarget products included:")
for product in final_df['Product'].unique():
    count = len(final_df[final_df['Product'] == product])
    print(f"  - {product}: {count:,} complaints")
print(f"\nAverage narrative length: {final_df['cleaned_length'].mean():.0f} characters")
print(f"Ready for embedding and vector store creation! 🚀")